In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from pathlib import Path
import pandas as pd
from tqdm import tqdm
import json
import numpy as np
import pickle

import matplotlib.pyplot as plt
import seaborn as sns
from openslide import OpenSlide

In [3]:
from probreg import cpd
from probreg import callbacks
from probreg import transformation as tf

In [4]:
plt.rcParams.update({'font.size': 25})

patch_size = 1024

In [5]:
def add_help_fields(frame):
    
    frame["image_name_stem"] = [Path(image_name).stem for image_name in frame["image_name"]]
    
    frame["patient_id"] = [name.split("_")[2] for name in frame["image_name"]]

    frame["x1"] = [json.loads(vector.replace("\'","\""))['x1'] for vector in frame["vector"]]
    frame["y1"] = [json.loads(vector.replace("\'","\""))['y1'] for vector in frame["vector"]]

    frame["x2"] = [json.loads(vector.replace("\'","\""))['x2'] for vector in frame["vector"]]
    frame["y2"] = [json.loads(vector.replace("\'","\""))['y2'] for vector in frame["vector"]]

    frame["center_x"] = [x1 + ((x2-x1) / 2) for x1, x2 in zip(frame["x1"], frame["x2"])]
    frame["center_y"] = [y1 + ((y2-y1) / 2) for y1, y2 in zip(frame["y1"], frame["y2"])]
    
    frame["center"] = [[center_x, center_y] for center_x, center_y in zip(frame["center_x"], frame["center_y"])]

    frame["anno_width"] = [x2-x1 for x1, x2 in zip(frame["x1"], frame["x2"])]
    frame["anno_height"]= [y2-y1 for y1, y2 in zip(frame["y1"], frame["y2"])]
    
    return frame

In [6]:
gt_annotations = add_help_fields(pd.read_csv("GT.csv"))
gt_annotations.head()

,scanner,image_id,image_name,image_type,image_width,image_height,id,vector,unique_identifier,annotation_type,...,patient_id,x1,y1,x2,y2,center_x,center_y,center,anno_width,anno_height
0,2.0HT,10410,N2_CCMCT_380609B_1.ndpi,CCMCT,147456,107008,2755406,"{'x1': 20354, 'x2': 20408, 'y1': 16752, 'y2': ...",95183f6e-0a00-4de4-9c97-fedfec655a27,384,...,380609B,20354,16752,20408,16806,20381.0,16779.0,"[20381.0, 16779.0]",54,54
1,2.0HT,10410,N2_CCMCT_380609B_1.ndpi,CCMCT,147456,107008,2755407,"{'x1': 20256, 'x2': 20310, 'y1': 35448, 'y2': ...",cc3def2b-5631-4619-ab08-51f2a69c667d,385,...,380609B,20256,35448,20310,35502,20283.0,35475.0,"[20283.0, 35475.0]",54,54
2,2.0HT,10410,N2_CCMCT_380609B_1.ndpi,CCMCT,147456,107008,2755408,"{'x1': 20445, 'x2': 20499, 'y1': 54139, 'y2': ...",2d3de917-e55a-470f-8af2-10eb49b69687,386,...,380609B,20445,54139,20499,54193,20472.0,54166.0,"[20472.0, 54166.0]",54,54
3,2.0HT,10410,N2_CCMCT_380609B_1.ndpi,CCMCT,147456,107008,2755409,"{'x1': 26495, 'x2': 26549, 'y1': 71854, 'y2': ...",611f9d35-6cdb-42ef-bc38-2f44c9aaa257,387,...,380609B,26495,71854,26549,71908,26522.0,71881.0,"[26522.0, 71881.0]",54,54
4,2.0HT,10410,N2_CCMCT_380609B_1.ndpi,CCMCT,147456,107008,2755410,"{'x1': 35443, 'x2': 35497, 'y1': 90091, 'y2': ...",bf3ef0d3-5153-4693-96ba-96047cb3ae24,388,...,380609B,35443,90091,35497,90145,35470.0,90118.0,"[35470.0, 90118.0]",54,54


In [7]:
results, quality = [], []


for patient_id in tqdm(gt_annotations["patient_id"].unique()):
    
    patient_annos = gt_annotations[gt_annotations["patient_id"] == patient_id]
    
    source_annos = patient_annos[patient_annos["scanner"] == "Aperio"]
    source_anno = source_annos.iloc[0]
    
    for scanner in ['2.0HT', 'Axio', 'S210']:
        
        target_annos = patient_annos[patient_annos["scanner"] == scanner]
        target_anno = target_annos.iloc[0]
        
        source = np.array([a for a in source_annos["center"]])
        target = np.array([a for a in target_annos["center"]])

        tf_param, sigma2, q = cpd.registration_cpd(source, target, 'affine')
        
        mpp_x_scale = tf_param.b[0][0]
        mpp_y_scale = tf_param.b[1][1]
        
        
        target_scanner = target_anno.scanner 
        image_id = target_anno.image_id 
        image_name = target_anno.image_name 
        image_width, image_height = target_anno.image_width, target_anno.image_height
        
        image_type = "CCMCT" if "CCMCT" in image_name else "Cyto"
        
        quality.append([patient_id, source_anno.image_name, target_anno.image_name, 
                        int(q), int(sigma2), image_type, tf_param.b, tf_param.t])
        
        for id, source_anno in source_annos.iterrows():          
            
            # trans center
            box_1 = [source_anno.center_x, source_anno.center_y]
            trans_box_1 = tf_param.transform([box_1])[0]
            
            # trans width and hight
            new_width, new_height = source_anno.anno_width * mpp_x_scale, source_anno.anno_height * mpp_y_scale
            
            trans_box = [trans_box_1[0], trans_box_1[1], new_width, new_height]
            
            new_x1 = int(trans_box[0] - trans_box[2] // 2)
            new_y1 = int(trans_box[1] - trans_box[3] // 2)
            new_x2 = int(trans_box[0] + trans_box[2] // 2)
            new_y2 = int(trans_box[1] + trans_box[3] // 2)
            
            vector = {'x1': new_x1, 'x2': new_x2, 'y1': new_y1, 'y2': new_y2}
            
            row = [target_scanner, image_id, image_name, image_type, image_width, image_height,
                    vector, source_anno.unique_identifier, source_anno.annotation_type, 
                   source_anno.type_name, "GT_Registration"]
            
            results.append(row)

quality = pd.DataFrame(quality, columns=["patient_id", "source", "target","q", "sigma2", "image_type", "b", "t"])

results = pd.DataFrame(results, columns=["scanner", "image_id", "image_name", "image_type", "image_width", 
                                         "image_height", "vector", "unique_identifier", "annotation_type", "type_name", "method"])
results.head()

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 18.09it/s]


,scanner,image_id,image_name,image_type,image_width,image_height,vector,unique_identifier,annotation_type,type_name,method
0,2.0HT,10410,N2_CCMCT_380609B_1.ndpi,CCMCT,147456,107008,"{'x1': 20353, 'x2': 20407, 'y1': 16756, 'y2': ...",95183f6e-0a00-4de4-9c97-fedfec655a27,384,L0,GT_Registration
1,2.0HT,10410,N2_CCMCT_380609B_1.ndpi,CCMCT,147456,107008,"{'x1': 20253, 'x2': 20307, 'y1': 35451, 'y2': ...",cc3def2b-5631-4619-ab08-51f2a69c667d,385,L1,GT_Registration
2,2.0HT,10410,N2_CCMCT_380609B_1.ndpi,CCMCT,147456,107008,"{'x1': 20444, 'x2': 20498, 'y1': 54140, 'y2': ...",2d3de917-e55a-470f-8af2-10eb49b69687,386,L2,GT_Registration
3,2.0HT,10410,N2_CCMCT_380609B_1.ndpi,CCMCT,147456,107008,"{'x1': 26498, 'x2': 26552, 'y1': 71856, 'y2': ...",611f9d35-6cdb-42ef-bc38-2f44c9aaa257,387,L3,GT_Registration
4,2.0HT,10410,N2_CCMCT_380609B_1.ndpi,CCMCT,147456,107008,"{'x1': 35440, 'x2': 35494, 'y1': 90089, 'y2': ...",bf3ef0d3-5153-4693-96ba-96047cb3ae24,388,L4,GT_Registration


In [8]:
results.to_csv("GT_Registration.csv", index=False)

In [9]:
quality

,patient_id,source,target,q,sigma2,image_type,b,t
0,380609B,A_CCMCT_380609B_1.svs,N2_CCMCT_380609B_1.ndpi,81,9,CCMCT,"[[1.1186536632777717, 0.003364233571722426], [...","[3030.451844627314, 3177.244143202617]"
1,380609B,A_CCMCT_380609B_1.svs,Z_CCMCT_380609B_1.tif,99,22,CCMCT,"[[1.1488890207406737, 0.005123966423343056], [...","[-1354.022303960126, 5255.945589512419]"
2,380609B,A_CCMCT_380609B_1.svs,N1_CCMCT_380609B_1.ndpi,79,8,CCMCT,"[[1.1476324941359377, -0.0010810197656771624],...","[2179.2892938877485, 4412.670006468063]"
3,518711B,A_CCMCT_518711B_1.svs,N2_CCMCT_518711B_1.ndpi,70,6,CCMCT,"[[1.1187191648757138, 0.00233285589007412], [-...","[1350.7098265751483, -465.6055849419936]"
4,518711B,A_CCMCT_518711B_1.svs,Z_CCMCT_518711B_1.tif,113,34,CCMCT,"[[1.1489909985565694, 0.015551755777784181], [...","[-3296.556960266418, -1216.4159854188765]"
5,518711B,A_CCMCT_518711B_1.svs,N1_CCMCT_518711B_1.ndpi,73,7,CCMCT,"[[1.147761289790462, -0.002118994954319657], [...","[394.96459003779455, 1059.3086949124845]"
6,183715A,A_CCMCT_183715A_1.svs,N2_CCMCT_183715A_1.ndpi,76,9,CCMCT,"[[1.1186968402482949, 0.0028714968325571975], ...","[465.9556085169461, 48.39405524612812]"
7,183715A,A_CCMCT_183715A_1.svs,Z_CCMCT_183715A_1.tif,89,15,CCMCT,"[[1.1489451665210357, 0.006513295621034563], [...","[-1991.4333699630224, -2304.599148558482]"
8,183715A,A_CCMCT_183715A_1.svs,N1_CCMCT_183715A_1.ndpi,73,7,CCMCT,"[[1.1477388122958607, -0.0014625339539216886],...","[3320.5873129679603, 5853.156087568088]"
9,29609B,A_CCMCT_29609B_1.svs,N2_CCMCT_29609B_1.ndpi,66,7,CCMCT,"[[1.1185165106728572, 0.0035524480645407973], ...","[9405.640020125487, -3274.577670741164]"


In [12]:
qm = quality[quality["target"] == "N1_CCMCT_183715A_1.ndpi"].iloc[0]
qm.b

array([[ 1.14773881e+00, -1.46253395e-03],
       [ 1.04505463e-03,  1.14744824e+00]])

In [13]:
qm.t

array([3320.58731297, 5853.15608757])